# Population Data from CSV

This notebooks reads sample population data from `data/atlantis.csv` and plots it using Matplotlib. Edit `data/atlantis.csv` and re-run this cell to see how the plots change!

In [1]:
pip install "ray[rllib]" tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 18.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.8/316.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.7/225.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 44.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 26.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 836.9/836.9 kB 11.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.9 MB/s

In [2]:
pip install swig

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.2 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install gymnasium[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 9.9 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 39.8 MB/s eta 0:00:0000:0100:01
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp310-cp310-linux_x86_64.whl size=3139665 sha256=f9f7ad96b9ac3f5cfc5e0fda8b58c8355a7b6782bae55ec330da450224d85726
  Stored in directory: /home/codespace/.cache/pip/wheels/db/8f/6a/eaaadf056fba10a98d986f6dce954e6201ba3126926fc5ad9e
Successfully built box2d-py

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install wandb

In [ ]:
import ray

ray.init()

In [15]:
from ray import tune

tune.run("PPO",
         config={"env": "CartPole-v1",
                 "evaluation_interval": 2,
                 "evaluation_num_episodes": 20,
                 "num_workers": 1,
                 # other configurations go here, if none provided, then default configurations will be used
                 },
         local_dir="CartPole-v1",
         checkpoint_freq=2
         )

2023-07-30 13:51:37,308	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
/home/codespace/.python/current/lib/python3.10/site-packages/ray/tune/tune.py:258: UserWarning: Passing a `local_dir` is deprecated and will be removed in the future. Pass `storage_path` instead or set the `RAY_AIR_LOCAL_CACHE_DIR` environment variable instead.
  warnings.warn(


2023-07-30 13:51:37,379	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-07-30 13:51:37,379	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.evaluation(evaluation_num_episodes=..)` has been deprecated. Use `AlgorithmConfig.evaluation(evaluation_duration=.., evaluation_duration_unit='episodes')` instead. This will raise an error in the future!
(pid=12451) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=12451) 2023-07-30 13:51:47,565	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. 

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,timers
PPO_CartPole-v1_33eb6_00000,76000,"{'ObsPreprocessorConnector_ms': 0.005317211151123047, 'StateBufferConnector_ms': 0.004159688949584961, 'ViewRequirementAgentConnector_ms': 0.2982957363128662}","{'num_env_steps_sampled': 76000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 76000, 'num_agent_steps_trained': 0}",{},476.42,{},500,476.42,207,8,"{'learner': {'__all__': {'num_agent_steps_trained': 128.0, 'num_env_steps_trained': 4000.0, 'total_loss': 9.750986611665185}, 'default_policy': {'total_loss': 9.750986611665185, 'policy_loss': -0.0038914912274039823, 'vf_loss': 9.754842504001122, 'vf_loss_unclipped': 893.3997122107793, 'vf_explained_var': -0.08397272413473393, 'entropy': 0.5292539340155974, 'mean_kl_loss': 0.0038003752765098714, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.004687500186264515}}, 'num_env_steps_sampled': 76000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 76000, 'num_agent_steps_trained': 0}",76000,0,76000,4000,264.523,0,0,0,0,1,0,0,0,"{'cpu_util_percent': 32.66190476190476, 'ram_util_percent': 83.71428571428571}",{},{},{},"{'mean_raw_obs_processing_ms': 0.5030812238499134, 'mean_inference_ms': 1.6563421000351797, 'mean_action_processing_ms': 0.12444791957644523, 'mean_env_wait_ms': 0.0639249175561622, 'mean_env_render_ms': 0.0}","{'episode_reward_max': 500.0, 'episode_reward_min': 207.0, 'episode_reward_mean': 476.42, 'episode_len_mean': 476.42, 'episode_media': {}, 'episodes_this_iter': 8, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 302.0, 500.0, 500.0, 500.0, 500.0, 500.0, 350.0, 207.0, 429.0, 500.0, 500.0, 500.0, 325.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 454.0, 500.0, 231.0, 379.0, 500.0, 500.0, 500.0, 500.0, 457.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 324.0, 402.0, 500.0, 500.0, 500.0, 439.0, 500.0, 500.0, 500.0, 425.0, 500.0, 500.0, 363.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 314.0, 500.0, 415.0, 500.0, 500.0, 500.0, 391.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 435.0, 500.0, 500.0], 'episode_lengths': [500, 500, 500, 500, 500, 500, 500, 302, 500, 500, 500, 500, 500, 350, 207, 429, 500, 500, 500, 325, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 454, 500, 231, 379, 500, 500, 500, 500, 457, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 324, 402, 500, 500, 500, 439, 500, 500, 500, 425, 500, 500, 363, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 314, 500, 415, 500, 500, 500, 391, 500, 500, 500, 500, 500, 500, 435, 500, 500]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.5030812238499134, 'mean_inference_ms': 1.6563421000351797, 'mean_action_processing_ms': 0.12444791957644523, 'mean_env_wait_ms': 0.0639249175561622, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.005317211151123047, 'StateBufferConnector_ms': 0.004159688949584961, 'ViewRequirementAgentConnector_ms': 0.2982957363128662}}","{'training_iteration_time_ms': 15123.362, 'sample_time_ms': 9437.681, 'synch_weights_time_ms': 3

Exception ignored in: <function RecordVideo.__del__ at 0x7f96b17191b0>
Traceback (most recent call last):
  File "/home/codespace/.python/current/lib/python3.10/site-packages/gymnasium/wrappers/record_video.py", line 211, in __del__
    self.close_video_recorder()
  File "/home/codespace/.python/current/lib/python3.10/site-packages/gymnasium/wrappers/record_video.py", line 180, in close_video_recorder
    self.video_recorder.close()
  File "/home/codespace/.python/current/lib/python3.10/site-packages/gymnasium/wrappers/monitoring/video_recorder.py", line 158, in close
    clip.write_videofile(self.path)
  File "/home/codespace/.local/lib/python3.10/site-packages/decorator.py", line 232, in fun
    for i, extra in enumerate(extras):
  File "/home/codespace/.python/current/lib/python3.10/site-packages/moviepy/decorators.py", line 54, in requires_duration
    return f(clip, *a, **k)
  File "/home/codespace/.local/lib/python3.10/site-packages/decorator.py", line 232, in fun
    for i, extr

Moviepy - Building video /workspaces/codespaces-jupyter/notebooks/somovieb/rl-video-episode-0.mp4.
Moviepy - Writing video /workspaces/codespaces-jupyter/notebooks/somovieb/rl-video-episode-0.mp4

Moviepy - Building video /workspaces/codespaces-jupyter/notebooks/somovieb/rl-video-episode-0.mp4.
Moviepy - Writing video /workspaces/codespaces-jupyter/notebooks/somovieb/rl-video-episode-0.mp4

Moviepy - Building video /workspaces/codespaces-jupyter/notebooks/somovieb/rl-video-episode-0.mp4.
Moviepy - Writing video /workspaces/codespaces-jupyter/notebooks/somovieb/rl-video-episode-0.mp4

Moviepy - Building video /workspaces/codespaces-jupyter/notebooks/somovieb/rl-video-episode-0.mp4.
Moviepy - Writing video /workspaces/codespaces-jupyter/notebooks/somovieb/rl-video-episode-0.mp4



2023-07-30 13:59:55,517	WARNING tune.py:192 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2023-07-30 14:00:05,538	INFO tune.py:1148 -- Total run time: 508.23 seconds (498.14 seconds for the tuning loop).
2023-07-30 14:00:05,540	WARNING tune.py:1163 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)


In [16]:
from ray.rllib.algorithms.algorithm import Algorithm

algo = Algorithm.from_checkpoint("/workspaces/codespaces-jupyter/notebooks/CartPole-v1/PPO/PPO_CartPole-v1_33eb6_00000_0_2023-07-30_13-51-37/checkpoint_000006")

2023-07-30 14:00:46,185	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-07-30 14:00:46,188	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
/home/codespace/.python/current/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:484: RayDeprecationWarning: Thi

In [ ]:
pip install moviepy

In [17]:
from gymnasium.wrappers.record_video import RecordVideo

try:
    import gymnasium as gym

    gymnasium = True
except Exception:
    import gym

    gymnasium = False

from ray.rllib.algorithms.ppo import PPOConfig

env_name = gym.make("BipedalWalker-v3", render_mode="rgb_array")
env = RecordVideo(env_name, "somovieb2")
algo = PPOConfig().environment("BipedalWalker-v3").rollouts(num_rollout_workers=1).build()

episode_reward = 0
terminated = truncated = False

if gymnasium:
    obs, info = env.reset()
else:
    obs = env.reset()

while not terminated and not truncated:
    action = algo.compute_single_action(obs)
    if gymnasium:
        obs, reward, terminated, truncated, info = env.step(action)
    else:
        obs, reward, terminated, info = env.step(action)
    episode_reward += reward

2023-07-30 14:01:03,925	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-07-30 14:01:03,927	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
/home/codespace/.python/current/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:484: RayDeprecationWarning: Thi

Moviepy - Building video /workspaces/codespaces-jupyter/notebooks/somovieb2/rl-video-episode-0.mp4.
Moviepy - Writing video /workspaces/codespaces-jupyter/notebooks/somovieb2/rl-video-episode-0.mp4



TypeError: must be real number, not NoneType